In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import wrds
pd.set_option('display.max_columns', 200)


In [2]:
#conn=wrds.Connection()

## Compustat

In [3]:
# # Compustat
# comp = conn.raw_sql("""
#                     select gvkey, datadate, at, pstkl, txditc,
#                     pstkrv, seq, pstk
#                     from comp.funda
#                     where indfmt='INDL' 
#                     and datafmt='STD'
#                     and popsrc='D'
#                     and consol='C'
#                     and datadate >='01/01/1959'
#                     """, date_cols=['datadate'])
# comp.to_csv('../data/wrds_comp_funda.csv')
comp = pd.read_csv('../data/wrds_comp_funda.csv', infer_datetime_format= True, parse_dates=['datadate'])

comp.rename(columns={'datadate':'date'}, inplace=True)

# Bring each date to its month end date - this is to ensure when we join DFs that they are successful
comp['date'] = comp['date'] + MonthEnd(0)

# Filter data to only include July 1963 to December 1993 (inclusive of both dates)
comp = comp[(comp['date']>='1963-01-01') & (comp['date']<='1993-12-31')]

# create preferrerd stock - used for market cap calculation
comp['ps'] = np.where(comp['pstkrv'].isnull(), comp['pstkl'], comp['pstkrv'])
comp['ps'] = np.where(comp['ps'].isnull(), comp['pstk'], comp['ps'])
comp['ps'] = np.where(comp['ps'].isnull(), 0,comp['ps'])
comp['txditc'] = comp['txditc'].fillna(0)

# create book equity
# BE is the COMPUSTAT book value of stockholders' equity, plus balance sheet deferred taxes and 
# investment tax credit (if available), minus the book value of preferred stock.
# (source: Multifactor Explanations of Asset Pricing Anomalies)

comp['be'] = comp['seq'] + comp['txditc'] - comp['ps']
comp['be'] = np.where(comp['be']>0, comp['be'], np.nan)

# number of years in Compustat
comp = comp.sort_values(by = ['gvkey','date'])
comp['count'] = comp.groupby(['gvkey']).cumcount()

comp = comp[['gvkey','date','be','count']]

comp['gvkey'] = comp.gvkey.astype('int32')

comp.head()

,gvkey,date,be,count
2,1000,1963-12-31,0.561,0
3,1000,1964-12-31,0.627,1
4,1000,1965-12-31,0.491,2
5,1000,1966-12-31,0.834,3
6,1000,1967-12-31,0.744,4


## CRSP

In [4]:
# # CRSP Block   
# # sql similar to crspmerge macro
# crsp_m = conn.raw_sql("""
#                       select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
#                       a.ret, a.retx, a.shrout, a.prc
#                       from crsp.msf as a
#                       left join crsp.msenames as b
#                       on a.permno=b.permno
#                       and b.namedt<=a.date
#                       and a.date<=b.nameendt
#                       where a.date between '01/01/1959' and '12/31/2017'
#                       and b.exchcd between 1 and 3
#                       """, date_cols=['date']) 
# crsp_m.to_csv('../data/wrds_crsp_msenames.csv')
crsp_m = pd.read_csv('../data/wrds_crsp_msenames.csv', infer_datetime_format= True, parse_dates=['date'])

crsp_m.drop(columns='Unnamed: 0', inplace=True)

# Filter data to only include July 1963 to December 1993 (inclusive of both dates)
crsp_m = crsp_m[(crsp_m['date']>='1963-01-01') & (crsp_m['date']<='1993-12-31')]
crsp_m.dtypes

# change variable format to int
crsp_m[['permco','permno','shrcd','exchcd']] = crsp_m[['permco','permno','shrcd','exchcd']].astype(int)

# Line up date to be end of month
crsp_m['date'] = crsp_m['date'] + MonthEnd(0)

# Only firms with ordinary common equity (as classified by CRSP) are included in the tests. 
# This means that ADR's, REIT's, and units of beneficial interest are excluded.
# WRDS provides the following definition of shrcd, "SHRCD is a two-digit code describing the type of shares 
# traded. The first digit describes the type of security traded." Ordinary Common Shares are defined by a 
# a 1 in the first digit. 

#Filter DF so only ordinary common shares are includes
crsp_m = crsp_m[(crsp_m['shrcd']==10) | (crsp_m['shrcd']==11)]

crsp_m.head()

,permno,permco,date,shrcd,exchcd,ret,retx,shrout,prc
1,10001,7953,1991-03-31,11,3,-0.011538,-0.025641,1054.0,9.500
2,10001,7953,1991-04-30,11,3,0.039474,0.039474,1054.0,-9.875
3,10001,7953,1991-05-31,11,3,0.000000,0.000000,1054.0,-9.875
4,10001,7953,1991-06-30,11,3,0.078481,0.063291,1073.0,10.500
5,10001,7953,1991-07-31,11,3,-0.035714,-0.035714,1073.0,10.125


In [5]:
# add delisting return - this is needed to incorporate the return for various events that
# cause a stock to be delisted such as merger or go-private transaction
# dlret  =  conn.raw_sql("""
#                      select permno, dlret, dlstdt 
#                      from crsp.msedelist
#                      """, date_cols = ['dlstdt'])
# dlret.to_csv('../data/wrds_dlret.csv')

dlret = pd.read_csv('../data/wrds_dlret.csv', infer_datetime_format= True, parse_dates=['dlstdt'],\
                   usecols=['permno', 'dlret', 'dlstdt'])

# Filter data to only include July 1963 to December 1993 (inclusive of both dates)
dlret = dlret[(dlret['dlstdt']>='1963-01-01') & (dlret['dlstdt']<='1993-12-31')]

dlret.permno = dlret.permno.astype(int)
dlret['dlstdt'] = dlret['dlstdt'] + MonthEnd(0)

dlret.rename(columns={'dlstdt': 'date'}, inplace=True)

# Line up date to be end of month
dlret['date'] = dlret['date'] + MonthEnd(0)

dlret.head()

,permno,dlret,date
0,10000,0.000000,1987-06-30
4,10005,0.125000,1991-07-31
5,10006,0.035629,1984-06-30
6,10007,-0.133333,1990-10-31
7,10008,0.009369,1988-11-30


In [6]:
crsp = pd.merge(crsp_m, dlret, how = 'left', on = ['permno','date'])
crsp['dlret'] = crsp['dlret'].fillna(0)
crsp['ret'] = crsp['ret'].fillna(0)

# retadj factors in the delisting returns
crsp['retadj'] = (1+crsp['ret'])*(1+crsp['dlret'])-1

# calculate market equity
crsp['me'] = crsp['prc'].abs() * crsp['shrout'] 
crsp = crsp.drop(['dlret','prc','shrout'], axis = 1)
crsp = crsp.sort_values(by = ['date','permco','me'])

crsp.head()

,permno,permco,date,shrcd,exchcd,ret,retx,retadj,me
517428,28820,6,1963-01-31,10,2,0.148649,0.148649,0.148649,12213.4375
517794,29161,64,1963-01-31,10,2,0.166667,0.166667,0.166667,9625.0000
241105,17670,74,1963-01-31,10,1,0.128289,0.128289,0.128289,40945.6250
263873,18702,267,1963-01-31,10,1,0.021277,0.021277,0.021277,36648.0000
1165936,58149,281,1963-01-31,10,1,-0.033887,-0.036545,-0.033887,38860.0000


In [7]:
### Aggregate Market Cap 
# sum of me across different permno belonging to same permco on a given date
crsp_summe = crsp.groupby(['date','permco'])['me'].sum().reset_index()

# join with sum of me to get the correct market cap info
crsp.drop(columns='me', inplace=True)

crsp = crsp.merge(crsp_summe, how = 'left', on = ['date','permco'])

# sort by permno and date and also drop duplicates
crsp = crsp.drop_duplicates(['permco', 'date'])

crsp.head()

,permno,permco,date,shrcd,exchcd,ret,retx,retadj,me
0,28820,6,1963-01-31,10,2,0.148649,0.148649,0.148649,12213.4375
1,29161,64,1963-01-31,10,2,0.166667,0.166667,0.166667,9625.0000
2,17670,74,1963-01-31,10,1,0.128289,0.128289,0.128289,40945.6250
3,18702,267,1963-01-31,10,1,0.021277,0.021277,0.021277,36648.0000
4,58149,281,1963-01-31,10,1,-0.033887,-0.036545,-0.033887,38860.0000


## CCM Linking Table

In [8]:
# CCM Block
# ccm=conn.raw_sql("""
#                   select gvkey, lpermno as permno, linktype, linkprim, 
#                   linkdt, linkenddt
#                   from crsp.ccmxpf_linktable
#                   where substr(linktype,1,1)='L'
#                   and (linkprim ='C' or linkprim='P')
#                   """, date_cols=['linkdt', 'linkenddt'])


ccm = pd.read_csv('../data/linking_table_crsp_compustat.csv',\
                            usecols=['gvkey', 'LPERMNO', 'LINKTYPE', 'LINKPRIM', 'LINKDT', 'LINKENDDT'],\
                           infer_datetime_format= True, parse_dates=['LINKENDDT'])

ccm.rename(columns={'LPERMNO': 'permno'}, inplace=True)

# Filter LINKTYPE to only include the most accurate links
dlret = ccm[(ccm['LINKTYPE']=='LC') | (ccm['LINKTYPE']=='LU')]

# Convert linkenddt to datetime
ccm.columns= ccm.columns.str.lower()
ccm = ccm[~ccm.linkenddt.str.contains('E')]
ccm = ccm[~ccm.linkenddt.isnull()]
ccm = ccm[~ccm.linkdt.isnull()]

# Convert columns to datetime format
ccm['linkdt'] = ccm.linkdt.astype('int32').astype('datetime64[ns]').dt.date
ccm['linkenddt'] = pd.to_datetime(ccm.linkenddt, format="%Y-%m-%d")

# if linkenddt is missing then set to today date
ccm['linkenddt'] = ccm['linkenddt'].fillna(pd.to_datetime('today'))

ccm.head()

,gvkey,linkprim,linktype,permno,linkdt,linkenddt
0,1000,P,NU,NaN,1970-01-01,1970-11-12
1,1000,P,LU,25881.0,1970-01-01,1978-06-30
2,1000,C,NU,NaN,1970-01-01,1970-09-29
3,1001,P,LU,10015.0,1970-01-01,1986-07-31
4,1001,C,NU,NaN,1970-01-01,1983-09-19


**Merge Compustat and CCM**

In [9]:
print(len(ccm), "<<< length of ccm")
comp_ccm = comp.merge(ccm, how='left', on=['gvkey'])
print(len(comp_ccm), "<<< length of comp_ccm")

# set link date bounds
comp_ccm = comp_ccm[(comp_ccm['date']>=comp_ccm['linkdt']) & (comp_ccm['date']<=comp_ccm['linkenddt'])]
comp_ccm = comp_ccm[['gvkey','permno', 'date','be', 'count']]

# Lag Compustat data 6 months (recall, FF lag all accounting data 6 months)
# "To ensure that the accounting variables are known before the returns they are used to explain, 
# we match the accounting data for all fiscal yearends in calendar year t - 1 (1962-1989) with the 
# returns for July of year t to June of t + 1. The 6-month (minimum) gap between fiscal yearend and the return
# tests is conservative." (Source: "The Cross-Section of Expected Stock Returns" Fama-French (1992))

comp_ccm['date + 6'] = comp_ccm['date'] + MonthEnd(0) + MonthEnd(6) # bring all dates to monthend then add 6 months
comp_ccm.drop(columns='date', inplace=True)
comp_ccm.head()

78922 <<< length of ccm
571368 <<< length of comp_ccm


,gvkey,permno,be,count,date + 6
22,1000,25881.0,10.544,7,1971-06-30
25,1000,25881.0,8.382,8,1972-06-30
28,1000,25881.0,7.309,9,1973-06-30
31,1000,25881.0,8.798,10,1974-06-30
34,1000,25881.0,8.279,11,1975-06-30


**Merge CRSP with the newly formed comp_ccm df**

In [10]:
# join comp_ccm and crsp
crsp_comp = crsp.merge(comp_ccm, how='inner', left_on=['permno', 'date'], right_on=['permno', 'date + 6'])

crsp_comp = crsp_comp.drop_duplicates(['permco', 'date'])
crsp_comp['year'] = crsp_comp.date.dt.year

# Organize columns
crsp_comp = crsp_comp.loc[: ,['permno', 'permco', 'gvkey','date', 'date + 6', 'shrcd', 'exchcd','retadj','me',\
                              'be', 'count']]
crsp_comp.head()

,permno,permco,gvkey,date,date + 6,shrcd,exchcd,retadj,me,be,count
0,41910,2966,6982,1970-07-31,1970-07-31,10,2,0.193115,27821.875,12.598,4
1,36134,6232,9287,1970-07-31,1970-07-31,10,2,0.135135,19493.250,10.963,7
2,68195,7404,11486,1970-07-31,1970-07-31,11,1,0.137056,177324.000,104.807,6
3,44548,20106,1594,1970-07-31,1970-07-31,11,2,-0.080000,8262.750,4.670,7
4,45583,20199,1655,1970-07-31,1970-07-31,11,2,-0.070423,45028.500,12.753,6


In [11]:
# DATA VALIATION - ENSURE THERE ARE NO DUPLICATE PERMCO AND DATE ROWS
temp = crsp_comp.groupby(['permco', 'date'])['retadj'].count().reset_index()
temp[temp.duplicated(['permco', 'date'])] # this should return a blank df

,permco,date,retadj


In [12]:
# Create BE/ME variable
crsp_comp.loc[:, 'BE/ME'] = crsp_comp.loc[:, 'be'] / crsp_comp.loc[: ,'me']
crsp_comp = crsp_comp[~crsp_comp['BE/ME'].isnull()]

# "We do not use negative BE firms, which are rare prior to 1980, when calculating the breakpoints for BE/ME 
# or when forming the size-BE/ME portfolios." (source: Multifactor Explanations of Asset Pricing Anomalies)
#crsp_comp = crsp_comp[crsp_comp['BE/ME']>=0]

In [13]:
# Create month column and then filter on June data. This used to create the size portfolios
crsp_comp['month'] = crsp_comp['date + 6'].dt.month
# find the fiscal year, used to groupby and find the quintiles
crsp_comp['fiscal year'] = crsp_comp['date'].map(lambda x: x.year if x.month <= 6 else x.year-1)


crsp_comp_jun = crsp_comp.loc[crsp_comp['month']==6, ['permco', 'fiscal year', 'me','BE/ME']]

In [14]:
crsp_comp_jun['ME Quantiles'] = pd.qcut(crsp_comp_jun['me'], q=[0, 0.20, 0.40, 0.60, 0.8, 1],\
                                          labels=[1,2,3,4,5])

crsp_comp_jun['BE/ME Quantiles'] = pd.qcut(crsp_comp_jun['BE/ME'], q=[0, 0.20, 0.40, 0.60, 0.8, 1],\
                                          labels=[1,2,3,4,5])

crsp_comp_jun.drop(columns=['me', 'BE/ME'], inplace=True)

print(len(crsp_comp), "<<< length of crsp_comp BEFORE merge")
crsp_comp = crsp_comp.merge(crsp_comp_jun, how='left', on=['permco', 'fiscal year'])
print(len(crsp_comp), "<<< length of crsp_comp AFTER merge")

# DATA VALIDATION - THIS SHOULD RETURN 0
# crsp_comp_jun[((crsp_comp_jun['ME Quantiles'].isnull()) | (crsp_comp_jun['BE/ME Quantiles'].isnull())) &\
#          (crsp_comp_jun['month']==6)]

85326 <<< length of crsp_comp BEFORE merge
85326 <<< length of crsp_comp AFTER merge


In [15]:
print(len(crsp_comp[crsp_comp['BE/ME Quantiles'].isnull()]))
print(len(crsp_comp[crsp_comp['ME Quantiles'].isnull()]))
#crsp_comp[crsp_comp['ME Quantiles'].isnull()]

crsp_comp = crsp_comp[~crsp_comp['BE/ME Quantiles'].isnull()]
crsp_comp = crsp_comp[~crsp_comp['ME Quantiles'].isnull()]

41426
41426


In [16]:
crsp_comp = crsp_comp[~(crsp_comp['BE/ME Quantiles'].isnull())]

In [17]:
crsp_comp['ME Quantiles'] = crsp_comp['ME Quantiles'].astype(str)
crsp_comp['BE/ME Quantiles'] = crsp_comp['BE/ME Quantiles'].astype(str)
crsp_comp.loc[: , 'Portfolio'] = 'ME' + crsp_comp['ME Quantiles'] + ' ' 'BE/ME' + crsp_comp['BE/ME Quantiles']

# crsp_comp.drop(columns=['ME Quantiles', 'BE/ME Quantiles', 'index', 'permno', 'permco', 'gvkey',\
#                         'shrcd', 'exchcd', 'count'], inplace=True)
crsp_comp.head()

,permno,permco,gvkey,date,date + 6,shrcd,exchcd,retadj,me,be,count,BE/ME,month,fiscal year,ME Quantiles,BE/ME Quantiles,Portfolio
649,41558,1453,4202,1971-06-30,1971-06-30,10,2,-0.100000,1408.500,2.876,7,0.002042,6,1971,1,5,ME1 BE/ME5
650,45102,1728,4641,1971-06-30,1971-06-30,11,1,-0.009231,86388.750,53.052,6,0.000614,6,1971,3,2,ME3 BE/ME2
651,31931,1848,4750,1971-06-30,1971-06-30,10,2,0.008984,16205.250,13.130,4,0.000810,6,1971,2,3,ME2 BE/ME3
652,43414,1897,5005,1971-06-30,1971-06-30,10,1,-0.101695,8870.875,6.463,7,0.000729,6,1971,1,3,ME1 BE/ME3
653,33400,2562,6448,1971-06-30,1971-06-30,10,2,0.087302,22365.250,5.198,7,0.000232,6,1971,2,1,ME2 BE/ME1


In [18]:
crsp_comp[crsp_comp.duplicated(['date + 6', 'gvkey'])]

,permno,permco,gvkey,date,date + 6,shrcd,exchcd,retadj,me,be,count,BE/ME,month,fiscal year,ME Quantiles,BE/ME Quantiles,Portfolio
23099,29787,23650,1884,1978-06-30,1978-06-30,10,3,0.216216,9000.000,2.151,4,0.000239,6,1978,1,1,ME1 BE/ME1
26702,29787,23650,1884,1979-06-30,1979-06-30,10,3,0.160000,5800.000,2.710,5,0.000467,6,1979,1,2,ME1 BE/ME2
30253,29787,23650,1884,1980-06-30,1980-06-30,10,3,0.071429,6000.000,3.561,6,0.000593,6,1980,1,2,ME1 BE/ME2
58311,11338,9185,14162,1987-06-30,1987-06-30,11,3,-0.096774,10500.000,3.463,1,0.000330,6,1987,2,1,ME2 BE/ME1
62650,11338,9185,14162,1988-06-30,1988-06-30,11,3,-0.175000,22683.375,6.913,2,0.000305,6,1988,2,1,ME2 BE/ME1


In [19]:
# groupby each portfolio on each month and calculate the total market equity
# this will be used to calculate the value weighted return

temp = crsp_comp.groupby(['Portfolio', 'fiscal year'])['me'].sum().reset_index().rename(columns={'me': 'Total ME'})
print(len(crsp_comp), "<<< length of crsp_comp BEFORE merge")
crsp_comp = crsp_comp.merge(temp, how='left', on=['Portfolio', 'fiscal year'])
print(len(crsp_comp), "<<< length of crsp_comp AFTER merge")

#crsp_comp.drop(columns='year', inplace=True)

crsp_comp.loc[: , 'wtd return'] = (crsp_comp.loc[: , 'me'] / crsp_comp.loc[: , 'Total ME'])\
                                    * crsp_comp.loc[: , 'retadj']

43900 <<< length of crsp_comp BEFORE merge
43900 <<< length of crsp_comp AFTER merge


In [20]:
crsp_comp['yearmonth'] =  crsp_comp['date'].dt.year.astype(str) + crsp_comp['date'].dt.month.astype(str)
crsp_comp.date.min()

Timestamp('1971-06-30 00:00:00')

In [21]:
100*pd.DataFrame(crsp_comp.groupby('Portfolio')['wtd return'].mean().reset_index()['wtd return'].values.reshape(5,5))

,0,1,2,3,4
0,0.020611,0.038495,0.012532,-0.015621,-0.015862
1,0.030196,0.023129,0.006427,0.002488,-0.017073
2,0.027564,0.018987,0.010987,0.002611,-0.013502
3,0.043233,0.025811,0.017205,0.007118,-0.020160
4,0.035179,0.026577,0.020227,0.010311,0.002248


**DON'T RUN THIS ONE**

In [22]:
100*pd.DataFrame(crsp_comp.groupby('Portfolio')['wtd return'].mean()\
                 .reset_index()['wtd return'].values.reshape(5,5))

,0,1,2,3,4
0,0.020611,0.038495,0.012532,-0.015621,-0.015862
1,0.030196,0.023129,0.006427,0.002488,-0.017073
2,0.027564,0.018987,0.010987,0.002611,-0.013502
3,0.043233,0.025811,0.017205,0.007118,-0.020160
4,0.035179,0.026577,0.020227,0.010311,0.002248


## Create Fama-French Portfolio

In [ ]:
# Form Fama French Factors

# function to calculate value weighted return
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan

In [ ]:
# value-weigthed return
vwret = ccm4.groupby(['jdate','szport','bmport']).apply(wavg, 'retadj','wt')\
            .to_frame().reset_index().rename(columns={0: 'vwret'})
vwret['sbport'] = vwret['szport'] + vwret['bmport']

# firm count
vwret_n = ccm4.groupby(['jdate','szport','bmport'])['retadj'].count().reset_index()\
            .rename(columns={'retadj':'n_firms'})
vwret_n['sbport'] = vwret_n['szport'] + vwret_n['bmport']
vwret.head(5)

In [ ]:
# transpose
ff_factors = vwret.pivot(index='jdate', columns='sbport', values='vwret').reset_index()
ff_nfirms = vwret_n.pivot(index='jdate', columns='sbport', values='n_firms').reset_index()

ff_factors.head()

In [ ]:
# create SMB and HML factors
ff_factors['WH'] = (ff_factors['BH'] + ff_factors['SH']) / 2
ff_factors['WL'] = (ff_factors['BL'] + ff_factors['SL']) / 2
ff_factors['WHML'] = ff_factors['WH'] - ff_factors['WL']

ff_factors['WB'] = (ff_factors['BL'] + ff_factors['BM'] + ff_factors['BH']) / 3
ff_factors['WS'] = (ff_factors['SL'] + ff_factors['SM'] + ff_factors['SH']) / 3
ff_factors['WSMB'] = ff_factors['WS'] - ff_factors['WB']
ff_factors = ff_factors.rename(columns={'jdate':'date'})

# n firm count - the number of firms in each bucket (e.g. BH, BL, BM, SH, etc)
ff_nfirms['H'] = ff_nfirms['SH'] + ff_nfirms['BH']
ff_nfirms['L'] = ff_nfirms['SL'] + ff_nfirms['BL']
ff_nfirms['HML'] = ff_nfirms['H'] + ff_nfirms['L']

ff_nfirms['B'] = ff_nfirms['BL'] + ff_nfirms['BM'] + ff_nfirms['BH']
ff_nfirms['S'] = ff_nfirms['SL'] + ff_nfirms['SM'] + ff_nfirms['SH']
ff_nfirms['SMB'] = ff_nfirms['B'] + ff_nfirms['S']
ff_nfirms['TOTAL'] = ff_nfirms['SMB']
ff_nfirms = ff_nfirms.rename(columns={'jdate':'date'})

## Compare with Fama French

In [ ]:
# Compare With FF 
#_ff  =  conn.get_table(library = 'ff', table = 'factors_monthly')
_ff = pd.read_csv('../data/fama_french_wrds.csv',infer_datetime_format= True, parse_dates=['date'])
_ff.dtypes

In [ ]:

_ff = _ff[['date','smb','hml']]
_ff['date'] = _ff['date'] + MonthEnd(0)

_ffcomp  =  pd.merge(_ff, ff_factors[['date','WSMB','WHML']], how='inner', on=['date'])
_ffcomp70 = _ffcomp[_ffcomp['date'] >= '01/01/1970']
print(stats.pearsonr(_ffcomp70['smb'], _ffcomp70['WSMB'])) # WSMB is our calculation, smb is the "official" number
print(stats.pearsonr(_ffcomp70['hml'], _ffcomp70['WHML'])) # WHML is our calculation, hml is the "official" number

In [ ]:
plt.figure(figsize=(16,12))
plt.suptitle('Comparison of Results', fontsize=20)

ax1 = plt.subplot(211)
ax1.set_title('SMB', fontsize=15)
ax1.set_xlim(_ffcomp['date'].min(), _ffcomp['date'].max())
ax1.plot(_ffcomp['date'], _ffcomp['WSMB'], 'b-')
ax1.plot(_ffcomp['date'], _ffcomp['smb'], 'r--')
ax1.legend(('smb','WSMB'), loc='upper right', shadow=True)

ax2 = plt.subplot(212)
ax2.set_title('HML', fontsize=15)
ax2.plot(_ffcomp['date'], _ffcomp['WHML'], 'b-')
ax2.plot(_ffcomp['date'], _ffcomp['hml'], 'r-')

ax2.set_xlim(_ffcomp['date'].min(), _ffcomp['date'].max())
ax2.legend(('hml','WHML'), loc='upper right', shadow=True)

plt.subplots_adjust(top=0.92, hspace=0.2)

plt.show()

## Appendix 

### Compustat Definitions:  
**Table Name:**  
Compustat Fundamentals Annual - comp.funda  

- **PSTKRV:** Preferred Stock Redemption Value	
    - This item represents the total dollar value of the net number of preferred shares outstanding multiplied by the voluntary liquidation or redemption value per share?whichever is greater.
- **PSTKL:**  Preferred Stock Liquidating Value
PSTK -- Preferred/Preference Stock (Capital) - Total
    - This item represents the net number of preferred shares at year-end multiplied by the par or stated value per share as presented in the company's Balance Sheet.
- **SEQ:**  Stockholders' Equity - Total	
    - This item represents the common and preferred shareholders' interest in the company.
- **TXDITC:**  Deferred Taxes and Investment Tax Credit	
    - This item represents the accumulated tax deferrals due to timing differences between the reporting of revenues and expenses for financial statements and tax forms and investment tax credit.

### CRSP Definitions:
**Table Name:**  
CRSP Monthly Stock File - crsp.msf  

- **shrcd:** Share Code  
    - Security type code (SHRCD) has two digits, with each digit representing a specific piece of information. The first defines the security type; the second provides more detailed information about the type of security traded. For example, a SHRCD of 10 or 11 would represent U.S. common stocks.  
- **exchd:** Exchange Code  
    - EXCHCD is a code indicating the exchange on which a security is listed. Normal exchange codes are respectively 1,2, and 3 for NYSE, AMEX and the Nasdaq Stock MarketSM. An exchange code of zero indicates that a security is either trading on an unknown exchange, or is temporarily not trading at all. 
- **ret:** Holding Period Return  
    - A return is the change in the total value of an investment in a common stock over some period of time per dollar of initial investment. RET(I) is the return for a sale on day I. It is based on a purchase on the most recent time previous to I when the se curity had a valid price. Usually, this time is I - 1.  
- **retx:** Holding Period Return without Dividends  
    - RETX contains returns without dividends. Ordinary dividends and certain other regularly taxable dividends are excluded from the returns calculation. The formula is the same as for RET except d(t) is usually 0.  
- **shrout**: Number of Shares Outstanding  
    - SHROUT is the number of publicly held shares, recorded in thousands  
- **prc:** Price 
    - Prc is the closing price or the negative bid/ask average for a trading day. If the closing price is not available on any given trading day, the number in the price field has a negative sign to indicate that it is a bid/ask average and not an actual closing price. Please note that in this field the negative sign is a symbol and that the value of the bid/ask average is not negative.


**Table Name:**  
CRSP Delisting Return - crsp.msedelist 

- **dlstdt:** Delist Date  
- **dlret:** Delisting Return
    - DLRET is the return of the security after it is delisted. It is calculated by comparing a value after delisting against the price on the security's last trading date. The value after delisting can include a delisting price or the amount from a final distribution. If NEXTDT is nonzero, it can be used as the effective date of DLRET  

### Linking Table
**Table Name:**  
CRSP Compustat Linking Table - crsp.ccmxpf_linktable

- **Linktype:**   
    - LINKTYPE is the link type code describing the connection between CRSP and Compustat data. Special attention should be paid to this variable to avoid double counting caused by multiple links.  
- **Linkprim:**  
    - LINKPRIM clarifies the link's relationship to Compustat's marked primary security within the related range. "P" indicates a primary link marker, as identified by Compustat in monthly security data. "C" indicates a primary link marker, as identified by CRSP to resolve ranges of overlapping or missing primary markers from Compustat in order to produce one primary security throughout the company history. "J" indicates a joiner secondary issue of a company, identified by Compustat in monthly security data.  
- **Linkdt:**  
    - LINKDT is a calendar date marking the first effective date of the current link. If the link was valid before CRSP's earliest record, LINKDT is set to be SAS missing code ".B".  
- **Linkenddt:**  
    -LINKENDDT is the last effective date of the link record. It uses the SAS missing code ".E" if a link is still valid.  
- **Lpermno:**  
    - LPERMNO is missing if no link exists for a given GVKEY. It is up to the user to decide on how to handle overlaps and soft links.  